# Evaluating narrowband SETI end-to-end detection performance
This notebook does the following:
1) Inputs a single RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)

A RAW file with multiple simultaneous drifting signals may be input to examine how SNR depends on drift rate. This may be generated separately by the notebook "00_multichirp_raw_file_gen.ipynb".

This notebook may be called by another notebook to sweep parameters and compare detection performance (detection SNR values)

This script is set up to look in a backup directory (raw_backup_base_dir + raw_backup_subdir) if the raw file is not found in the primary one (raw_dir).

In [1]:
import sys
import os

%matplotlib inline
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'medium',
          'figure.figsize': (10,6),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

import src.plot_fns as pltg             # generic plot fns
import src.plot_h5_psd_sg1 as plt_h5    # blimpy-based plot fns

import math
import numpy as np
from astropy import units as u
import blimpy as bl
import time
import pandas

from pathlib import Path

# sys.path.append(os.getenv('SETIGEN_PATH'))

# import setigen as stg

output_dir = os.getenv('DATADIR') + '/'
if not os.path.isdir(output_dir[0:-1]):
    os.system('mkdir '+output_dir[0:-1])

raw_dir = os.getenv('RAWDIR') + '/'
if not os.path.isdir(raw_dir[0:-1]):
    os.system('mkdir '+raw_dir[0:-1])

raw_backup_base_dir = os.getenv('RAW_BACKUP_BASE_DIR') + '/'

sg_dir = os.getenv('SGDIR') + '/'
if not os.path.isdir(sg_dir[0:-1]):
    os.system('mkdir '+sg_dir[0:-1])

output_dir = sg_dir

def db(x):
    """ Convert linear value to dB value """
    return 10*np.log10(np.abs(x.astype(np.float64))+1e-20)

def wt_avg(snr_db,drift,sigma_drift):
    """ Probability-weighted SNR average according to drift rate """
    wt_avg1 = np.zeros(len(sigma_drift))
    sigma_drift1 = sigma_drift.astype(np.float64)
    for idx,sigma1 in enumerate(sigma_drift1):
        w = np.exp(-0.5*(drift.astype(np.float64)/sigma1)**2)
        wt_avg1[idx] = np.dot(snr_db,w)/np.sum(w)
    return wt_avg1

def file_name_mjd(raw_file_stem):
    # specific to guppi file naming conventions
    stem_parts = raw_file_stem.split('_')
    if (len(stem_parts)<4):
         mjd_int = 99999
    elif (stem_parts[1]=='guppi'):
        mjd_int = int(stem_parts[2])
    elif (stem_parts[2]=='guppi'):
        mjd_int = int(stem_parts[3])
    else:
        mjd_int = 99999
    #print(mjd_int)   
    return mjd_int


In [ ]:
try:
    parameters_are_undefined
except NameError:
    parameters_are_undefined = True     
    print('Parameters are undefined, using defaults\n')


In [3]:
# os.environ['SETIGEN_ENABLE_GPU'] = '1'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
# Sometimes it can be necessary to re-run this command for plots to show automatically
%matplotlib inline

#### Parameter setup for seticore and turboseti runs
Need to be sure proper seticore2 branch is set up and compiled (e.g. "git checkout sc2", "git status", "meson compile"), and indicated here

In [ ]:
# test_case = 'iac24'
# test_case = 'iac24managed'
# test_case = 'sc0'
# test_case = 'sc1'
# test_case = 'sc1a'
test_case = 'sc2'
# test_case = 'sc2a'
# test_case = 'sc2b'

print(f'{test_case=}')

if ('iac' in test_case):
    sc_app_name = 'seticore'
else:
    sc_app_name = 'seticore2'

if ('sc0' in test_case):
    search_z_threshold = 10
else:
    search_z_threshold = 20

min_abs_drift_Hz_sec = .04

print(f'{sc_app_name=}, {search_z_threshold=}')

In [ ]:
if parameters_are_undefined:
    display_figs01 = True
    plot_dets1 = True
    plot_dets2 = True
    verbose = True
    plot_snr_vs_f_zoom = False
    plot_spectra = False
    plot_snr_vs_f = False
    zoom_bw_MHz = 1.0
    do_fig_text = True
    do_profile = False

    run_turbo = False
    fb_ext = '.h5'

    search_max_drift = 10    # Hz/sec
    search_min_drift = -10   # ignored except for plotting

        
    if (1):
        raw_backup_subdir = 'raw_multichirp/'
        # raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec'  # seticore crashes n_sti=1
        raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec'
        # raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec'
        # raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec'
        # raw_file_stem = 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec'
        # raw_file_stem = 'gbt-chirp80-0.0000V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec-32'
        # raw_file_stem = 'gbt-chirp80-0.0000V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec-8'
        fine_fft_size = 1024*1024*2
        # fine_fft_size = 1024*1024
        # fine_fft_size = 1024*256
        # n_sti = 2
        n_sti = 16
        plot_spectra = True
        plot_snr_vs_f = False
        zoom_bw_MHz = .1
    elif(1):
        raw_backup_subdir = 'raw_multichirp/'
        # raw_file_stem = 'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-160.75sec'
        raw_file_stem = 'meerkat-chirp80-0.0040V-1502.30-1502.70-m10.0-10.0-Hzsec-321.49sec'
        fine_fft_size = 512*1024
        n_sti = 8
        plot_spectra = True
        plot_snr_vs_f = False
        zoom_bw_MHz = .1
    elif(1):
        raw_backup_subdir = 'raw_multichirp/'
        if (1):
            raw_file_stem = 'cosmic-chirp80-0.0040V-1401.70-1402.30-m10.0-10.0-Hzsec-67.11sec'
        else:
            raw_file_stem = 'cosmic-chirp80-0.0040V-1401.70-1402.30-m50.0-50.0-Hzsec-67.11sec'
            search_max_drift = 50    # Hz/sec
            search_min_drift = -50   # ignored except for plotting
   
        fine_fft_size = 512*1024
        n_sti = 1
        plot_spectra = True
        plot_snr_vs_f = False
        zoom_bw_MHz = .1
    elif(0):
        raw_backup_subdir = 'raw_multichirp/'
        raw_file_stem = 'vlass-chirp80-0.0040V-1401.80-1402.40-m10.0-10.0-Hzsec-8.00sec'
        fine_fft_size = 128*1024
        n_sti = 1
        plot_spectra = True
        plot_snr_vs_f = False
        zoom_bw_MHz = .1
    elif(1):
        raw_backup_subdir = 'raw_voyager/'
        raw_file_stem = 'blc23_guppi_59046_80036_DIAG_VOYAGER-1_0011' # 24.4 dB, -.279 Hz/sec
        # raw_file_stem = 'blc23_guppi_59046_80354_DIAG_VOYAGER-1_0012' # no det for 0000
        # raw_file_stem = 'blc23_guppi_59046_80672_DIAG_VOYAGER-1_0013' # 19.51 dB, -.504 Hz/sec
        # raw_file_stem = 'blc23_guppi_59046_80989_DIAG_VOYAGER-1_0014' # no det for 0000
        # raw_file_stem = 'blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015' # 23.7 dB, -.504 Hz/sec
        #raw_file_stem = 'blc23_guppi_59046_81628_DIAG_VOYAGER-1_0016' # no det for 0000
        #raw_file_stem = 'blc00_guppi_59046_80036_DIAG_VOYAGER-1_0011'   # 11064-11251 MHz, not 8420
        #raw_file_stem = 'blc02_guppi_59046_80036_DIAG_VOYAGER-1_0011'  # 10688-10876 MHz, not 8420
        #raw_file_stem = 'blc2_2bit_guppi_57396_VOYAGER1_0004'  # can't read 2-bit samples
        #raw_file_stem = 'blc3_2bit_guppi_57396_VOYAGER1_0004'  # can't read 2-bit samples
        # raw_file_stem = 'blc2_2bit_guppi_57396_VOYAGER1_0006'  # can't read 2-bit samples
        # fine_fft_size = 1024*1024
        fine_fft_size = 1024*256
        n_sti = 4
        plot_spectra = False
        plot_snr_vs_f = False
    elif(0):
        raw_backup_subdir = 'raw_voyager/'
        raw_file_stem = 'blc3_guppi_57386_VOYAGER1_0004'  # 8568.75 Hz, not 8420
        # fine_fft_size = 1024*1024    # creates an error due to odd block size  
        # fine_fft_size = 1033216     # =1024*1009, believe it or not
        fine_fft_size = 1033216/2     # =512*1009, believe it or not
        n_sti = 2                   # n_sti=4 -> h5 size = 3.4 GB
        plot_spectra = False
        plot_snr_vs_f = False
    elif(0):
        raw_backup_subdir = 'raw_gbt_57388_HIP/'
        # raw_file_stem = 'blc1_guppi_57388_HIP113357_0010' # 1969 MHz
        # raw_file_stem = 'blc2_guppi_57388_HIP113357_0010' # 1781 MHz
        # raw_file_stem = 'blc3_guppi_57388_HIP113357_0010' # 1594 MHz
        # raw_file_stem = 'blc4_guppi_57388_HIP113357_0010' # 1406 MHz
        # raw_file_stem = 'blc5_guppi_57388_HIP113357_0010' # 1219 MHz
        raw_file_stem = 'blc6_guppi_57388_HIP113357_0010' # 1031 MHz
        # fine_fft_size = 1033216     # =1024*1009, believe it or not
        fine_fft_size = 1033216/2     # =512*1009, believe it or not
        n_sti = 2                   # n_sti=4 -> h5 size = 3.4 GB
        plot_spectra = False
        plot_snr_vs_f = False
        zoom_bw_MHz = 1.0
    elif(1):
        raw_backup_subdir = 'raw_gbt_59103_Kepler/'
        raw_file_stem = 'blc40_guppi_59103_01984_DIAG_KEPLER-160_0010' # 2158 MHz
        # raw_file_stem = 'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010' # 1970 MHz
        # raw_file_stem = 'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010' # 1782 MHz
        # raw_file_stem = 'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010' # 1595 MHz
        # raw_file_stem = 'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010' # 1408 MHz
        # raw_file_stem = 'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010' # 1220 MHz
        # raw_file_stem = 'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010' # 1033 MHz
        # raw_file_stem = 'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010' # 845 MHz
        # raw_file_stem = 'blc40_guppi_59103_01984_DIAG_KEPLER-160_0010.0001' # 2158 MHz
        fine_fft_size = 1024*512    
        n_sti = 2                   # n_sti=4 -> h5 size = 3.4 GB
        plot_spectra = False
        plot_snr_vs_f = False
        zoom_bw_MHz = 1.0       
    
    fig_dir = 'plots/'
    # #remove previous plots - only if parameters_are_undefined
    # if os.path.isdir(fig_dir[0:-1]):
    #     os.system('rm -rf '+fig_dir)

    isChirp = raw_file_stem.lower().find('chirp')>=0
    isVoyager = raw_file_stem.lower().find('voyager')>=0
    isguppi = raw_file_stem.lower().find('guppi')>=0

    drift_limit_nHz = 1.0  # assume 2 sigma limit
    f_sigma_drift = np.array([1.5, 3, 6])*1e9
    sigma_drift = drift_limit_nHz*f_sigma_drift/2.*1e-9
    print(f'sigma_drift = {sigma_drift}')

    delete_raw_file = False
    
raw_file_base_name = raw_dir + raw_file_stem + '.0000.raw'
raw_file_delete_spec = raw_dir + raw_file_stem + '.00*.raw'

print(raw_file_base_name)
if os.path.isfile(raw_file_base_name): 
    print('Raw file found in raw directory')
else:
    raw_file_backup_name = raw_backup_base_dir + raw_backup_subdir + raw_file_stem + '.0000.raw'
    raw_file_backup_spec = raw_backup_base_dir + raw_backup_subdir + raw_file_stem + '.00*.raw'
    if os.path.isfile(raw_file_backup_name): 
        print('Raw file found in backup directory, copying')
        if (1):     # single raw file .0000.raw
            print(raw_file_backup_name)
            os.system('cp '+raw_file_backup_name+' '+raw_dir)
            os.system('chmod 666 '+raw_dir+'*.raw')  # remove read-only spec from raw_dir
        else:       # multi raw file .00xx.raw
            os.system('ls '+raw_file_backup_spec)
            os.system('cp '+raw_file_backup_spec+' '+raw_dir)
            os.system('chmod 666 '+raw_dir+'*.raw')  # remove read-only spec from raw_dir
        os.system('ls -lsah '+raw_dir+'*.raw')
        print('Copy complete')
        delete_raw_file = True
    else:
        print('Raw file not found in backup directory, skip')
        print(raw_file_backup_name)
        # stop, or return to calling notebook/script
        assert(False)




In [7]:
mjd_int = file_name_mjd(raw_file_stem)
if (mjd_int < 58000):
    if (fine_fft_size%1009 > 0):
        print(f'Warning: {fine_fft_size=} doesn''t evenly divide block size for early raw files\n')


In [ ]:
print(raw_file_stem)
raw_parts_list = raw_file_stem.split('-')
print(raw_parts_list)

if (isChirp):
    telescope = raw_parts_list[0]
    signal_level = float(raw_parts_list[2][0:-1])
    n_chirp = int(raw_parts_list[1][5:])
    f_start1_MHz = float(raw_parts_list[3])
    f_stop1_MHz = float(raw_parts_list[4])
    if (raw_parts_list[5][0]=='m'):
        sig_min_drift = -float(raw_parts_list[5][1:])  # remove minus sign with 'm' for negative drift limits in raw file name
    else:
        sig_min_drift = float(raw_parts_list[5])
    if (raw_parts_list[6][0]=='m'):
        sig_max_drift = -float(raw_parts_list[6][1:])  # remove minus sign with 'm' for negative drift limits in raw file name
    else:
        sig_max_drift = float(raw_parts_list[6])
    print(f'{n_chirp = } {f_start1_MHz = } {f_stop1_MHz = } {sig_min_drift = } {sig_max_drift = }')
        

elif(isVoyager):
    signal_level = 0.
    n_chirp = 2
    if (0):
        delta_f1_MHz = 1.5
        f_start1_MHz = 8420.432 - delta_f1_MHz
        f_stop1_MHz = 8420.432 + delta_f1_MHz
    else:
        delta_f1_MHz = np.nan
        f_start1_MHz = np.nan
        f_stop1_MHz = np.nan

    sig_min_drift = search_min_drift
    sig_max_drift = search_med_drift
elif(isguppi):
    signal_level = 0.
    n_chirp = 2
    delta_f1_MHz = np.nan
    f_start1_MHz = np.nan
    f_stop1_MHz = np.nan
    
    sig_min_drift = search_min_drift
    sig_max_drift = search_med_drift
    

f_start_truth = np.linspace(f_start1_MHz*1e6,f_stop1_MHz*1e6,n_chirp)
drift_rate_truth = np.linspace(sig_min_drift,sig_max_drift,n_chirp)
f_start_incr = f_start_truth[1]-f_start_truth[0]
df_dt_incr = drift_rate_truth[1]-drift_rate_truth[0]

if verbose:
    print(f'{signal_level = }, {n_chirp = }')
    print(f'{f_start1_MHz = }, {f_stop1_MHz = }, { f_start_incr = }, {df_dt_incr = }')
    print(f'{f_start_truth = }')
    print(f'{drift_rate_truth = }')

 


#### Obtain run parameters from raw files and fine_fft_size and n_sti parameters

In [ ]:
import src.get_raw_info as raw
p = raw.get_run_params(raw_file_stem,raw_dir,fine_fft_size,n_sti)
print(p)
# exec(open("src/get_raw_info.py").read())
# copy p entries into local variables
%run -i "src/raw_info_script.py"

if (n_lti<=2):
    print(f'Note: {n_lti=}, skipping execution\n')
    # stop, or return to calling notebook/script
    assert(False)

if isguppi:
    telescope = telescop

In [ ]:
p = raw.get_run_params(raw_file_stem,raw_dir,1024*1024,n_sti)
p

In [ ]:
snr_input_db = 10*np.log10(signal_level**2 /2)  # 1/2 2 pols
snr_coarse_db = snr_input_db + 10*np.log10(n_coarse_channels)
snr_fine_db = snr_coarse_db + 10*np.log10(fine_fft_size)
snr_det_db = snr_fine_db + 5*np.log10(n_avg)

print('signal_level={0:6.4f}, Input SNR={1:4.2f} Coarse SNR={2:4.2f} Fine SNR={3:4.2f} Det SNR={4:4.2f} dB'.format(\
        signal_level,snr_input_db,snr_coarse_db,snr_fine_db,snr_det_db))

ref_snr_db = snr_det_db
print('signal_level={0:6.4f}, Est SNR={1:4.2f}, Reference SNR={2:4.2f} dB'.format(signal_level,snr_det_db,ref_snr_db))

if verbose:
    print('f_start_truth = ',f_start_truth*1e-6,' MHz')
    print('drift_rate_truth = ',drift_rate_truth,' Hz/sec')

# open directory for figures if necessary
if not os.path.isdir(fig_dir[0:-1]):
    os.system('mkdir '+fig_dir[0:-1])
# empty old figures

config_str = 'Fine FFT Size=%.0fK, Nsti=%d, Nlti=%.0f  %s'%(fine_fft_size/1024,n_sti,n_lti,test_case)
config_str2 = 'FineFFT-%.0fK-Nsti-%02d'%(fine_fft_size/1024,n_sti)
if verbose:print(config_str,'\n',config_str2)
fig_name_base = raw_file_stem + '-' + config_str2
if verbose: print(f'{fig_name_base = }')

if math.isnan(f_start1_MHz):
    fig_df = obs_bw_MHz*1e6
    fig_f_limits_MHz = [f_min_MHz , f_max_MHz]
    f_start1_MHz = f_min_MHz
    f_stop1_MHz = f_max_MHz
else:
    fig_df = (np.amax(f_start_truth)*1e-6 - np.amin(f_start_truth)*1e-6)*.25
    fig_f_limits_MHz = [np.amin(f_start_truth)*1e-6 - fig_df,np.amax(f_start_truth)*1e-6 + fig_df]
if verbose: print("figure freq limits MHz: ",fig_f_limits_MHz)

#### Generate spectrogram file using rawspec

In [ ]:
if (fb_ext == '.h5'):
    raw_call_string = f'rawspec -f {fine_fft_size} -t {n_sti} -j ' + raw_dir + raw_file_stem
    # raw_call_string = f'rawspec -f {fine_fft_size} -t {n_sti} -s 3 -n 1 -j ' + raw_dir + raw_file_stem
elif (fb_ext == '.fil'):
    raw_call_string = f'rawspec -f {fine_fft_size} -t {n_sti} -j ' + raw_dir + raw_file_stem
if verbose: print(raw_call_string + '\n')
t1 = time.time()
os.system(raw_call_string)
t_rawspec0 = time.time() - t1
t_rawspec = t_rawspec0/n_coarse_channels
print(f'\nRawspec {t_rawspec0:.2f} sec {t_rawspec:.2f} sec/coarse channel')

file_stats = os.stat(raw_file_base_name)
raw_size_MB0 = file_stats.st_size/1024/1024
raw_size_MB = raw_size_MB0/n_coarse_channels
if verbose: print(f'raw File size = {raw_size_MB0:6.0f} MB {raw_size_MB:6.0f} MB per Coarse Channel')


#### Augment file name of filterbank h5/fil file to include parameters (fft, n_sti)
Move to output_dir if required

In [ ]:
import glob
import os
h5_name_list = glob.glob(raw_dir + raw_file_stem + '.*' + fb_ext)
print(raw_dir + raw_file_stem + '.*' + fb_ext)
print(h5_name_list,'\n')

for i_name, h5_name in enumerate(h5_name_list):
    if (1):
        #use the same name to avoid excessive disk storage space (same file overwritten many times)
        new_h5_name = h5_name
    else:
        h5_parts = h5_name.split('.rawspec')
        print(f'{h5_parts = }')
        new_h5_name = h5_parts[0] + '-fft%.0fK-int%02d'%(fine_fft_size/1024,n_sti) + '.rawspec' + h5_parts[1]
    if (1):   # change output directory from raw_dir to sg_dir
        new_h5_name = output_dir + os.path.basename(new_h5_name)
    if (i_name==0):
        base_h5_name = new_h5_name
    if verbose: print(f'{new_h5_name = }')
    os.system('mv ' + h5_name + ' ' + new_h5_name)

base_dat_name = base_h5_name.split(fb_ext)[0] + '.dat'

if verbose:
    print(f'{base_h5_name = }')
    print(f'{base_dat_name = }')

print(new_h5_name)
file_stats = os.stat(new_h5_name)
h5_size_MB0 = file_stats.st_size/1024/1024
h5_size_MB = h5_size_MB0/n_coarse_channels
if verbose: print(f'H5 File size = {h5_size_MB0:6.0f} MB {h5_size_MB:6.0f} MB per Coarse Channel')

#### Plot spectra as required:

In [ ]:

if isChirp:
    
    import src.plot_h5_psd_sg1 as plt_h5
    
    if plot_spectra & (h5_size_MB<5000.0):

        %matplotlib inline

        plt_h5.plot_h5_sg1(base_h5_name,
                fig_f_limits_MHz=fig_f_limits_MHz,
                min_max_db=[],
                fig_title=raw_file_stem  + ' ' + config_str,
                display_fig=display_figs01,
                savfig_name=fig_dir+'01-'+fig_name_base+'-waterfall-'+test_case+'.png')
        
        

In [15]:
if isChirp:

    if plot_snr_vs_f  & (h5_size_MB<5000.0):

        %matplotlib inline

        plt_h5.plot_h5_psd1(base_h5_name,
                fig_f_limits_MHz=fig_f_limits_MHz,
                min_max_db=[0.,np.amax((40.,ref_snr_db))],
                fig_title=config_str,
                fig_text_list=[[.15,.85,raw_file_stem]],
                display_fig=display_figs01,
                savfig_name=fig_dir+'02-'+fig_name_base+'-psd-'+test_case+'.png')


#### Run turbo_seti if required

In [ ]:
if run_turbo:
    from turbo_seti.find_doppler.find_doppler import FindDoppler

    # Get rid of any pre-existing output files from a prior run.
    # for x_file in sorted(os.listdir(output_dir)):
    #     x_type = x_file.split('.')[-1]
    #     if x_type != 'h5':
    #         os.remove(output_dir + x_file)

    # Get ready for search by instantiating the doppler object.
    doppler = FindDoppler(base_h5_name,
                        max_drift = search_max_drift,
                        min_drift = min_abs_drift_Hz_sec,
                        snr = 10,       
                        out_dir = output_dir # This is where the turboSETI output files will be stored.
                        )

    t1 = time.time()
    doppler.search()
    t_turbo0 = time.time() - t1
    t_turbo = t_turbo0/n_coarse_channels
    print(f'\TurboSeti Search complete {t_turbo0:.2f} sec {t_turbo:.2f} sec/coarse channel')

    from turbo_seti.find_event.find_event import read_dat

print(base_dat_name)

In [ ]:
print(f'{output_dir = }')
print(f'{base_dat_name = }')
#os.system('ls '+output_dir+'*.dat')


In [ ]:
df_turbo = []
if run_turbo:
    from turbo_seti.find_event.find_event import read_dat
    df_turbo = read_dat(base_dat_name)
    f_start_turbo = df_turbo.values[:,5]*1e6
    drift_rate_turbo = df_turbo.values[:,1]
    snr_turbo_db = db(df_turbo.values[:,2])
    # for ff, dd, ss in zip(f_start_turbo*1e-6, drift_rate_turbo,snr_turbo_db):
    #        print('{0:8.2f} {1:8.2f} {2:8.2f}'.format(ff,dd,ss))
    ii = np.nonzero(np.abs(drift_rate_turbo)>df_dt_incr/2)
    snr_turbo_db_wavg = wt_avg(snr_turbo_db[ii],drift_rate_turbo[ii],sigma_drift)
    print(snr_turbo_db_wavg)

df_turbo


#### Run seticore to detect drifting tones and note execution time

In [ ]:
seticore_str = sc_app_name + ' ' + base_h5_name + \
    ' --max_drift='+ str(search_max_drift) + ' --min_drift=' + str(min_abs_drift_Hz_sec) + \
    ' --snr=' + str(search_z_threshold) + '  --output=' + output_dir + 'testout.dat'

#    | tee data/output.txt
if verbose: print(seticore_str)

t1 = time.time()
if do_profile:
    os.system('nsys profile ' + seticore_str + ' > seticore_text.out')
else:
    # os.system(seticore_str)
    os.system(seticore_str+ ' > seticore_text.out')

t_core0 = time.time() - t1
t_core = t_core0/n_coarse_channels
print(f'\nSeticore Search complete, {t_core0:.2f} sec {t_core:.2f} sec/coarse channel')


In [20]:
# sys.path.append(os.getenv('SETICORE_PY_PATH'))
# import viewer
# import numpy as np
# events = list(viewer.read_events(output_dir + 'testout.hits'))

In [ ]:
from turbo_seti.find_event.find_event import read_dat
df_core = read_dat(output_dir + 'testout.dat')
df_core

In [ ]:
f_start_core = df_core.values[:,5]*1e6
drift_rate_core = df_core.values[:,1]
snr_core_db = db(df_core.values[:,2])
# print(snr_core_db)
# for ff, dd, ssnr in zip(f_start_core*1e-6, drift_rate_core,snr_core_db):
#        print('{0:8.2f} {1:8.2f} {2:8.2f}'.format(ff,dd,ssnr))
ii = np.nonzero(np.abs(drift_rate_core)>df_dt_incr/2)
snr_core_db_wavg = wt_avg(snr_core_db[ii],drift_rate_core[ii],sigma_drift)

if verbose: print(f'{snr_core_db_wavg = }')

#### Zoom plots near center of detected frequencies

In [ ]:
if len(snr_core_db)>0:

    # Calculate the index of the median detection frequency
    sorted_indices = np.argsort(f_start_core)
    i_med = sorted_indices[len(f_start_core) // 2]
    snr_med = snr_core_db[i_med]
    f_start_med_MHz = f_start_core[i_med]*1e-6
    drift_rate_med = drift_rate_core[i_med]

    # Calculate the peak snr and associated freq/drift rate
    i_max = np.argmax(snr_core_db)
    snr_max = snr_core_db[i_max]
    f_start_max_MHz = f_start_core[i_max]*1e-6
    drift_rate_max = drift_rate_core[i_max]

else:
    snr_med = float("NaN")
    f_start_med_MHz = float("NaN")
    drift_rate_med = float("NaN")
    snr_max = float("NaN")
    f_start_max_MHz = float("NaN")
    drift_rate_max = float("NaN")

peak_string = f'{snr_med= :.2f} dB, {f_start_med_MHz = :.3f} Hz, {drift_rate_med = :.3f} Hz/sec'

if verbose:
    print(peak_string)

In [24]:
if plot_snr_vs_f_zoom  & (h5_size_MB0<5000.0):

        %matplotlib inline

        plt_h5.plot_h5_psd1(base_h5_name,
                fig_f_limits_MHz=np.add(f_start_med_MHz,[-zoom_bw_MHz/2,zoom_bw_MHz/2]),
                min_max_db=[0.,np.amax((40.,ref_snr_db))],
                fig_title=config_str,
                fig_text_list=[[.15,.85,raw_file_stem],[.15,.80,peak_string]],
                display_fig=display_figs01,
                savfig_name=fig_dir+'02z1-'+fig_name_base+'-psd-'+test_case+'.png')

zoom_bw_MHz = .005
if plot_snr_vs_f_zoom & (h5_size_MB0<5000.0):

        %matplotlib inline

        plt_h5.plot_h5_sg1(base_h5_name,
                fig_f_limits_MHz=np.add(f_start_med_MHz,[-zoom_bw_MHz/2,zoom_bw_MHz/2]),
                min_max_db=[],
                fig_title=raw_file_stem  + ' ' + config_str,
                display_fig=display_figs01,
                savfig_name=fig_dir+'02z2-'+fig_name_base+'-waterfall-'+test_case+'.png')


### Plot detection results

In [ ]:
if (plot_dets1 & isChirp):
    %matplotlib inline
    for i_plot in range(2):
        if (i_plot==0) & (not run_turbo): 
            continue
            
        fig = plt.figure(figsize=(10, 6))
        plt.subplot(2,1,1)
        if i_plot==0:
            plt.plot(f_start_truth*1e-6 , drift_rate_truth,'*',label='Reference')
            plt.plot(f_start_turbo*1e-6 , drift_rate_turbo,'r*',label='Turbo')
            plt.title(telescope.upper() + ' TurboSETI Drifting Tone Detections, ' + config_str)
            t_search = t_turbo
            snr_db_wavg = snr_turbo_db_wavg
        else:
            plt.plot(f_start_truth*1e-6 , drift_rate_truth,'*',label='Reference')
            plt.plot(f_start_core*1e-6 , drift_rate_core,'g*',label='Seticore')
            plt.title(telescope.upper() + ' Seticore Drifting Tone Detections, ' + config_str)
            t_search = t_core
            snr_db_wavg = snr_core_db_wavg
        
        print(np.array2string(snr_db_wavg,precision=2))
        print('E(SNR)    '+ np.array2string(snr_db_wavg,precision=2) + ' dB')
        t_total = t_rawspec + t_search

        plt.xlim(fig_f_limits_MHz[0],fig_f_limits_MHz[1])
        plt.ylim(search_min_drift-1,search_max_drift+1)
        #plt.xlabel('Frequency (MHz)')
        plt.ylabel('Drift Rate (Hz/sec)')
        plt.figtext(.15,.85,raw_file_stem)
        plt.figtext(.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 {h5_size_MB:3.0f} MB')
        plt.figtext(.15,.78,f'Rawspec {t_rawspec:5.2f} sec')
        plt.figtext(.15,.75,f'Search    {t_search:5.2f} sec')
        plt.figtext(.15,.72,f'Total       {t_total:5.2f} sec')
        plt.figtext(.15,.68,'E(SNR)    '+ np.array2string(snr_db_wavg,precision=1) + ' dB')
        plt.legend(loc='lower right')
        plt.grid()

        plt.subplot(2,1,2)
        
        if i_plot==0:
            plt.plot(f_start_turbo*1e-6 , snr_turbo_db,'r*',label='Turbo')
        else:
            plt.plot(f_start_core*1e-6 , snr_core_db,'g*',label='Seticore')

        plt.xlim(fig_f_limits_MHz[0],fig_f_limits_MHz[1])
        plt.ylim(5., np.amax((40.,5.*np.ceil(ref_snr_db/5)+5.)))
        plt.xlabel('Frequency (MHz)')
        plt.ylabel('SNR(dB)')
        # plt.figtext(.15,.42,peak_string)
        plt.legend(loc='lower right')
        plt.grid()

        if i_plot==0:
            plt.savefig(fig_dir+'04-' + fig_name_base+'-turbo-det-'+test_case+'.png',bbox_inches='tight')
        else:
            plt.savefig(fig_dir+'06-' + fig_name_base+'-seticore-det-'+test_case+'.png',bbox_inches='tight')

        if display_figs01:
            plt.show()
        else:
            plt.close(fig)




In [26]:
if plot_dets2 & run_turbo & isChirp:
        t_search = t_turbo
        t_total = t_rawspec + t_search
        snr_string ='E(SNR)    ' + np.array2string(snr_turbo_db_wavg,precision=1) + ' dB'
        
        pltg.plot_generic(x_data=[drift_rate_truth,drift_rate_turbo],
                y_data=[ref_snr_db*np.ones(np.size(drift_rate_truth)),snr_turbo_db],
                xy_markers = ['*','r*'],
                xy_legend = ['Reference','TurboSETI'],
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[5., np.amax((45.,5.*np.ceil(ref_snr_db/5)+5.))],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + ' TurboSETI Drifting Tone Detections, ' + config_str,
                fig_text_list=[[.15,.85,raw_file_stem],
                                [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 {h5_size_MB:3.0f} MB'],
                                [.15,.78,f'Rawspec {t_rawspec:5.2f} sec'],
                                [.15,.75,f'Search    {t_search:5.2f} sec'], 
                                [.15,.72,f'Total       {t_total:5.2f} sec'],
                                [.15,.68,snr_string]],
                legend_loc = 'lower right',
                display_fig=display_figs01,
                savfig_name=fig_dir+'05-' + fig_name_base+'-turbo-det-'+test_case+'.png')


In [ ]:

if plot_dets2 & isChirp:
        t_search = t_core
        t_total = t_rawspec + t_search
        snr_string ='E(SNR)    ' + np.array2string(snr_core_db_wavg,precision=2) + ' dB'
        
        pltg.plot_generic(x_data=[drift_rate_truth,drift_rate_core],
                y_data=[ref_snr_db*np.ones(np.size(drift_rate_truth)),snr_core_db],
                xy_markers = ['-','r*'],
                xy_legend = ['Reference','Seticore'],
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[5., np.amax((45.,5.*np.ceil(ref_snr_db/5)+5.))],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + ' Seticore Drifting Tone Detections, ' + config_str,
                fig_text_list=[[.15,.85,raw_file_stem],
                                [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 {h5_size_MB:3.0f} MB'],
                                [.15,.78,f'Rawspec {t_rawspec:5.2f} sec'],
                                [.15,.75,f'Search    {t_search:5.2f} sec'], 
                                [.15,.72,f'Total       {t_total:5.2f} sec'],
                                [.15,.68,snr_string]],
                legend_loc = 'lower right',
                display_fig=display_figs01,
                savfig_name=fig_dir+'07-' + fig_name_base+'-seticore-det-'+test_case+'.png')

if plot_dets2 & isChirp:
        t_search = t_core
        t_total = t_rawspec + t_search
        snr_string ='E(SNR)    ' + np.array2string(snr_core_db_wavg,precision=2) + ' dB'
        prob_weight = 30*np.exp(-np.square(np.outer(drift_rate_truth,np.reciprocal(sigma_drift)))/2.)
        prob_legend = ['']*len(sigma_drift)
        for i_cfreq,cfreq in enumerate(f_sigma_drift):
                prob_legend[i_cfreq] = f'Prob Weight {cfreq*1e-9:.1f} GHz'

        pltg.plot_generic(x_data=[drift_rate_truth,drift_rate_core,drift_rate_truth],
                y_data=[ref_snr_db*np.ones(np.size(drift_rate_truth)),snr_core_db,prob_weight],
                xy_markers = ['-','r*','--'],
                xy_legend = ['Reference','Seticore',prob_legend],
                x_limits=[sig_min_drift-1,sig_max_drift+1],
                y_limits=[0., np.amax((40.,5.*np.ceil(ref_snr_db/5)+5.))],
                x_label = 'Drift Rate (Hz/sec)',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + ' Seticore Drifting Tone Detections, ' + config_str,
                fig_text_list=[[.15,.85,raw_file_stem],
                                [.15,.82,f'raw {raw_size_MB:3.0f} MB, h5 {h5_size_MB:3.0f} MB'],
                                [.15,.78,f'Rawspec {t_rawspec:5.2f} sec'],
                                [.15,.75,f'Search    {t_search:5.2f} sec'], 
                                [.15,.72,f'Total       {t_total:5.2f} sec'],
                                [.15,.68,snr_string]],
                legend_loc = 'upper right',
                display_fig=display_figs01,
                savfig_name=fig_dir+'08-' + fig_name_base+'-seticore-det-'+test_case+'.png')




In [ ]:
print(f'fig 09: {plot_dets2=}, {isguppi=}, \n{fig_dir=}, {fig_name_base=}')
if plot_dets2 & isguppi:
        t_search = t_core
        t_total = t_rawspec + t_search
        snr_string =peak_string
        
        pltg.plot_generic(x_data=f_start_core*1e-6,y_data=snr_core_db,
                xy_markers = 'g*',
                xy_legend = 'Seticore',
                x_limits= [f_min_MHz,f_max_MHz],
                y_limits=[5., np.amax((40.,5.*np.ceil(ref_snr_db/5)+5.))],
                x_label = 'Frequency MHz',
                y_label = 'SNR(dB)',
                fig_title= telescope.upper() + ' Seticore Drifting Tone Detections, ' + config_str,
                fig_text_list=[[.15,.85,raw_file_stem],
                                [.15,.82,f'{f_min_MHz:.0f}-{f_max_MHz:.0f} MHz, {search_min_drift:.0f}->{search_max_drift:.0f} Hz/sec, {t_obs:.1f} sec'],
                                [.15,.79,f'raw {raw_size_MB:3.0f} MB, h5 {h5_size_MB:3.0f} MB'],
                                [.15,.75,f'Rawspec {t_rawspec:5.2f} sec'],
                                [.15,.72,f'Search    {t_search:5.2f} sec'], 
                                [.15,.69,f'Total       {t_total:5.2f} sec'],
                                [.15,.65,peak_string]],
                legend_loc = 'upper right',
                display_fig=display_figs01,
                savfig_name=fig_dir+'09-' + fig_name_base+'-seticore-det-'+test_case+'.png')


In [29]:
# p

In [30]:
# whos

In [31]:
# os.system("echo -ne '\007'")
# os.system("echo -ne '\a'")
# os.system("tput bel")
# print("\007")


In [ ]:
# Beep in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")